In [1]:
from custom_losses import dice
from eval_tools import *
from data_tools import *
from seq_net import *
from unet import UNet


import cv2
import json
import torch.nn as nn
import matplotlib.pyplot as plt

In [2]:
'''os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
torch.set_default_tensor_type('torch.cuda.FloatTensor')
device = torch.device("cuda:0")'''

'os.environ["CUDA_VISIBLE_DEVICES"] = "0" \ntorch.set_default_tensor_type(\'torch.cuda.FloatTensor\')\ndevice = torch.device("cuda:0")'

In [3]:
device = torch.device("cpu")

In [4]:
test_json = json.load(open('test.json'))
folder = '/mnt/LAIN_EXTERNAL2/waterpolo_anyagok/vizilabda_meccsek/frames'

# Evaluation of the sequential network

# Evaluation of the U-net

In [5]:
test_ds = dataset_with_mask(folder,test_json)
test_iterator = data.DataLoader(test_ds, 
                                batch_size = 8)

model = UNet(in_channels=3,
             out_channels=1,
             n_blocks=2,
             start_filters=32,
             activation='relu',
             normalization='batch',
             conv_mode='same',
             dim=2)
model.load_state_dict(torch.load('top_results/otf_model_net_epoch41_loss=0.4606550086411276.pt'))
model.to('cpu')

{'UNet': {'in_channels': 3, 'out_channels': 1, 'n_blocks': 2, 'start_filters': 32, 'activation': 'relu', 'normalization': 'batch', 'conv_mode': 'same', 'dim': 2, 'up_mode': 'transposed'}}

In [6]:
ROC_dict = {}
cumulative_fp_num = 0
for i, data in enumerate(test_iterator, 0):
    inputs, labels,fnames = data[0].cpu(), data[1].cpu(),data[2]
    outputs = model(inputs)
    outputs[outputs<0.15] = 0
    outputs[outputs>0.15] = 1
    '''true_pos_rate = tpr(labels,outputs)
    false_pos_rate = fpr(labels,outputs)
    ROC_dict[false_pos_rate] = true_pos_rate'''
    fpn = N_false_pos(labels,outputs)
    cumulative_fp_num += fpn
    
    if i%10 == 0:
        print("Checkpoint",i)
print(cumulative_fp_num/(8*i))

/home/csenge/.local/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Checkpoint 0
Checkpoint 10
Checkpoint 20
Checkpoint 30
Checkpoint 40
Checkpoint 50
Checkpoint 60
Checkpoint 70
Checkpoint 80
Checkpoint 90
Checkpoint 100
Checkpoint 110
Checkpoint 120
Checkpoint 130
Checkpoint 140
1.4335106382978724


In [7]:
myList = ROC_dict.items()
myList = sorted(myList)
x, y = zip(*myList)



plt.plot(x, y)
plt.xlabel('FP rate')
plt.ylabel('TP rate')
plt.title('ROC curve')
plt.show()

ValueError: not enough values to unpack (expected 2, got 0)

In [ ]:
print(ROC_dict)